<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Federated_Learning_(FL)_with_Differential_Privacy_(DP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np
import random

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc = nn.Linear(784, 10)

    def forward(self, x):
        return self.fc(x.view(-1, 784))

def local_training(data_loader, model, epochs=1):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    for epoch in range(epochs):
        for data, target in data_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
    return model

def add_noise_to_gradients(model, epsilon=0.1, clipping_norm=1.0):
    for param in model.parameters():
        if param.grad is not None:
            grad_norm = param.grad.norm()
            param.grad = param.grad / max(1, grad_norm / clipping_norm)  # Clip gradients
            noise = torch.randn_like(param) * epsilon
            param.grad += noise

def aggregate_models(global_model, local_models):
    global_dict = global_model.state_dict()
    for key in global_dict.keys():
        global_dict[key] = torch.mean(torch.stack([local_model.state_dict()[key] for local_model in local_models]), dim=0)
    global_model.load_state_dict(global_dict)
    return global_model

def validate(global_model, test_loader):
    global_model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = global_model(data)
            pred = output.argmax(dim=1)
            correct += (pred == target).sum().item()
            total += target.size(0)
    accuracy = 100.0 * correct / total
    return accuracy

# Dataset and loaders
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
dataset = datasets.MNIST('/tmp/MNIST', download=True, transform=transform)
client_datasets = [Subset(dataset, np.arange(i, len(dataset), 10)) for i in range(10)]
client_loaders = [DataLoader(ds, batch_size=32, shuffle=True) for ds in client_datasets]
test_loader = DataLoader(dataset, batch_size=64, shuffle=False)

global_model = SimpleNN()
local_models = [SimpleNN() for _ in range(10)]

# Federated learning rounds
for round in range(5):
    for i, loader in enumerate(client_loaders):
        local_models[i] = local_training(loader, local_models[i])
        add_noise_to_gradients(local_models[i])

    global_model = aggregate_models(global_model, local_models)
    test_accuracy = validate(global_model, test_loader)
    print(f'Round {round} completed, Test Accuracy: {test_accuracy:.2f}%')